# Idealista API JSON Files
Convert JSON files into one DataFrame to work with

In [1]:
# Load Google Drive
from google.colab import drive

# Mount Google Drive through authorization
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/...

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# Load all necessary libraries
import json, os
from pandas import json_normalize
import pandas as pd

All JSON files located here: https://drive.google.com/drive/folders/1O3q6C6gzfQo-X4ZoDVpPE8-r4l5766gt?usp=sharing

In [3]:
### CHANGE THIS PATHNAME TO THE ONE THAT CONTAINS ALL JSON FILES ###
path = '/content/drive/My Drive'
# Set Working Directory where all JSON files are located
os.chdir(path)
# View Working Directory
os.getcwd()

'/content/drive/My Drive'

In [4]:
# Empty list to store data
appended_data = []

# For loop to go through all files in the directory
for filename in os.listdir(path):
    # Condition to only open JSON files
    if filename.endswith('.json'):
        # Load all JSON files
        with open(os.path.join(path, filename)) as f:
            data = json.load(f)
    # Create DataFrame from normalized JSON dictionary
    df = pd.DataFrame.from_dict(json_normalize(data['elementList']))

    # Append the empty list to store the DataFrame data
    appended_data.append(df)

# Concatinate DataFrames
appended_data = pd.concat(appended_data)
df = appended_data
# Save DataFrame as a CSV
#df.to_csv('ALL-JSON-FILES.csv')

# Count number of variables (columns)
print('Number of variables:', len(df.columns))
# Count number of records (rows)
print('Number of records:', len(df.index))

# See IF a file has a wrong format
#print(f)

Number of variables: 40
Number of records: 16500


## Overview of the Data

In [5]:
# Preview "dirty" data
df.head()

,propertyCode,thumbnail,externalReference,numPhotos,price,propertyType,operation,size,exterior,rooms,bathrooms,address,province,municipality,district,country,latitude,longitude,showAddress,url,distance,hasVideo,status,newDevelopment,newDevelopmentFinished,priceByArea,hasPlan,has3DTour,has360,parkingSpace.hasParkingSpace,parkingSpace.isParkingSpaceIncludedInPrice,detailedType.typology,detailedType.subTypology,suggestedTexts.subtitle,suggestedTexts.title,neighborhood,floor,hasLift,parkingSpace.parkingSpacePrice,topNewDevelopment
0,[86350566],[https://img3.idealista.com/blur/WEB_LISTING/0...,[97123],[34],[612900],[chalet],[sale],[334],[False],[4],[3],"[Calle Gonzalo Jiménez de Quesada, 1]",[Madrid],[Boadilla del Monte],[Sector B],[es],[40.4136],[-3.9162],[True],[https://www.idealista.com/obra-nueva/86350566/],[18029],[False],[newdevelopment],[True],[False],[1835],[True],[False],[False],[True],[True],[chalet],[semidetachedHouse],"[Sector B, Boadilla del Monte]",[Chalet pareado en Calle Gonzalo Jiménez de Qu...,NaN,NaN,NaN,NaN,NaN
1,[88820218],[https://img3.idealista.com/blur/WEB_LISTING/0...,[POZ23040],[38],[1250000],[chalet],[sale],[396],[False],[5],[4],[barrio Monteclaro],[Madrid],[Pozuelo de Alarcón],[Urbanizaciones],[es],[40.4415],[-3.8361],[False],[https://www.idealista.com/inmueble/88820218/],[11572],[False],[good],[False],NaN,[3157],[False],[False],[False],NaN,NaN,[chalet],NaN,"[Monteclaro, Pozuelo de Alarcón]",[Chalet],[Monteclaro],NaN,NaN,NaN,NaN
2,[88536347],[https://img3.idealista.com/blur/WEB_LISTING/0...,[AS146617],[26],[289000],[flat],[sale],[44],[False],[1],[1],"[Calle de Pelayo, 26]",[Madrid],[Madrid],[Centro],[es],[40.4228],[-3.6984],[True],[https://www.idealista.com/inmueble/88536347/],[789],[True],[good],[False],NaN,[6568],[True],[True],[False],NaN,NaN,[flat],NaN,"[Chueca-Justicia, Madrid]","[Piso en Calle de Pelayo, 26]",[Chueca-Justicia],[3],[False],NaN,NaN
3,[88744592],[https://img3.idealista.com/blur/WEB_LISTING/0...,[CL147007],[57],[2900000],[chalet],[sale],[800],[False],[8],[7],[La Moraleja],[Madrid],[La Moraleja],[La Moraleja],[es],[40.5121],[-3.6235],[False],[https://www.idealista.com/inmueble/88744592/],[12572],[True],[good],[False],NaN,[3625],[True],[True],[True],[True],[True],[chalet],[independantHouse],"[La Moraleja, La Moraleja]",[Casa independiente],NaN,NaN,NaN,NaN,NaN
4,[87535127],[https://img3.idealista.com/blur/WEB_LISTING/0...,[AS143181],[26],[430000],[flat],[sale],[93],[True],[1],[2],"[Paseo del Prado, 14]",[Madrid],[Madrid],[Centro],[es],[40.4142],[-3.6944],[True],[https://www.idealista.com/inmueble/87535127/],[795],[True],[good],[False],NaN,[4624],[True],[True],[False],NaN,NaN,[flat],NaN,"[Huertas-Cortes, Madrid]","[Piso en Paseo del Prado, 14]",[Huertas-Cortes],[4],[True],NaN,NaN


In [6]:
# View all DataFrame column names
df.columns

Index(['propertyCode', 'thumbnail', 'externalReference', 'numPhotos', 'price',
       'propertyType', 'operation', 'size', 'exterior', 'rooms', 'bathrooms',
       'address', 'province', 'municipality', 'district', 'country',
       'latitude', 'longitude', 'showAddress', 'url', 'distance', 'hasVideo',
       'status', 'newDevelopment', 'newDevelopmentFinished', 'priceByArea',
       'hasPlan', 'has3DTour', 'has360', 'parkingSpace.hasParkingSpace',
       'parkingSpace.isParkingSpaceIncludedInPrice', 'detailedType.typology',
       'detailedType.subTypology', 'suggestedTexts.subtitle',
       'suggestedTexts.title', 'neighborhood', 'floor', 'hasLift',
       'parkingSpace.parkingSpacePrice', 'topNewDevelopment'],
      dtype='object')

In [7]:
# See Data Types
df.dtypes

propertyCode                                  object
thumbnail                                     object
externalReference                             object
numPhotos                                     object
price                                         object
propertyType                                  object
operation                                     object
size                                          object
exterior                                      object
rooms                                         object
bathrooms                                     object
address                                       object
province                                      object
municipality                                  object
district                                      object
country                                       object
latitude                                      object
longitude                                     object
showAddress                                   

## Cleaning and transformations

In [8]:
# STRINGS
for column in df:
    # Convert to string
    df[column] = df[column].astype(str)
    # Remove unnecessary characters
    df[column] = df[column].str.lstrip("['")
    df[column] = df[column].str.rstrip("]'")
    # Replace nans to NaNs (None)
    df[column] = df[column].replace(to_replace='nan', value=None)

# INTEGERS
for column in df[['propertyCode', 'numPhotos', 'price', 'priceByArea', 'rooms', 'bathrooms', 'distance', 'size', 'parkingSpace.parkingSpacePrice']]:
    # Convert to integers
    df[column] = pd.to_numeric(df[column], errors='coerce')
    
# TRUE/FALSE
for column in df[['exterior', 'showAddress', 'hasPlan', 'has3DTour', 'has360', 'hasVideo', 'hasLift', 'newDevelopment', 'parkingSpace.hasParkingSpace', 'parkingSpace.isParkingSpaceIncludedInPrice', 'newDevelopmentFinished', 'topNewDevelopment']]:
    # Capitalize TRUE and FALSE
    df[column] = df[column].str.upper()
    # Replace NaNs to FALSE
    df[column] = df[column].replace(to_replace='NAN', value='FALSE')
    # Convert TRUE (1) and FALSE (0) to integers
    (df[column] == 'TRUE').astype(int)

# Categorical variables (Label Encoding) ?
#for column in df[['status']]:
    #df['column'] = df['column'].astype('category')

# Drop unnecessary columns
df.drop(['thumbnail', 'showAddress', 'numPhotos', 'url', 'hasPlan', 'has3DTour', 'has360', 'hasVideo', 'externalReference', 'province', 'operation', 'country', 'detailedType.typology', 'suggestedTexts.subtitle', 'suggestedTexts.title', 'detailedType.subTypology'], axis=1, inplace=True)

# WHAT TO DO WITH THOSE ?
#df['neighborhood']
#df['municipality']
#df['district']
#df['address']

In [9]:
# See Data Types
df.dtypes

propertyCode                                    int64
price                                           int64
propertyType                                   object
size                                          float64
exterior                                       object
rooms                                           int64
bathrooms                                       int64
address                                        object
municipality                                   object
district                                       object
latitude                                       object
longitude                                      object
distance                                        int64
status                                         object
newDevelopment                                 object
newDevelopmentFinished                         object
priceByArea                                     int64
parkingSpace.hasParkingSpace                   object
parkingSpace.isParkingSpaceI

## Clean DataFrame

In [10]:
# Preview data
df.head()

,propertyCode,price,propertyType,size,exterior,rooms,bathrooms,address,municipality,district,latitude,longitude,distance,status,newDevelopment,newDevelopmentFinished,priceByArea,parkingSpace.hasParkingSpace,parkingSpace.isParkingSpaceIncludedInPrice,neighborhood,floor,hasLift,parkingSpace.parkingSpacePrice,topNewDevelopment
0,86350566,612900,chalet,334.0,FALSE,4,3,"Calle Gonzalo Jiménez de Quesada, 1",Boadilla del Monte,Sector B,40.4136,-3.9162,18029,newdevelopment,TRUE,FALSE,1835,TRUE,TRUE,nan,nan,FALSE,NaN,FALSE
1,88820218,1250000,chalet,396.0,FALSE,5,4,barrio Monteclaro,Pozuelo de Alarcón,Urbanizaciones,40.4415,-3.8361,11572,good,FALSE,FALSE,3157,TRUE,TRUE,Monteclaro,nan,FALSE,NaN,FALSE
2,88536347,289000,flat,44.0,FALSE,1,1,"Calle de Pelayo, 26",Madrid,Centro,40.4228,-3.6984,789,good,FALSE,FALSE,6568,TRUE,TRUE,Chueca-Justicia,3,FALSE,NaN,FALSE
3,88744592,2900000,chalet,800.0,FALSE,8,7,La Moraleja,La Moraleja,La Moraleja,40.5121,-3.6235,12572,good,FALSE,FALSE,3625,TRUE,TRUE,Chueca-Justicia,3,FALSE,NaN,FALSE
4,87535127,430000,flat,93.0,TRUE,1,2,"Paseo del Prado, 14",Madrid,Centro,40.4142,-3.6944,795,good,FALSE,FALSE,4624,TRUE,TRUE,Huertas-Cortes,4,TRUE,NaN,FALSE


In [11]:
# Total amount of unique Property Codes
total = len(df.index)
uniques = len(df['propertyCode'].unique())

print('There are {} unique properties out of {}.'.format(uniques, total))

There are 6572 unique properties out of 16500.


In [12]:
# Total number of duplicates
df.duplicated(subset = 'propertyCode', keep = 'first').sum()

9928

What is the difference between UNIQUE and DUPLICATES?

In [13]:
# Dropping ALL duplicte values
df.drop_duplicates(subset ='propertyCode', 
                     keep = 'first', inplace = True)

In [14]:
# Preview unique data
df.head()

,propertyCode,price,propertyType,size,exterior,rooms,bathrooms,address,municipality,district,latitude,longitude,distance,status,newDevelopment,newDevelopmentFinished,priceByArea,parkingSpace.hasParkingSpace,parkingSpace.isParkingSpaceIncludedInPrice,neighborhood,floor,hasLift,parkingSpace.parkingSpacePrice,topNewDevelopment
0,86350566,612900,chalet,334.0,FALSE,4,3,"Calle Gonzalo Jiménez de Quesada, 1",Boadilla del Monte,Sector B,40.4136,-3.9162,18029,newdevelopment,TRUE,FALSE,1835,TRUE,TRUE,nan,nan,FALSE,NaN,FALSE
1,88820218,1250000,chalet,396.0,FALSE,5,4,barrio Monteclaro,Pozuelo de Alarcón,Urbanizaciones,40.4415,-3.8361,11572,good,FALSE,FALSE,3157,TRUE,TRUE,Monteclaro,nan,FALSE,NaN,FALSE
2,88536347,289000,flat,44.0,FALSE,1,1,"Calle de Pelayo, 26",Madrid,Centro,40.4228,-3.6984,789,good,FALSE,FALSE,6568,TRUE,TRUE,Chueca-Justicia,3,FALSE,NaN,FALSE
3,88744592,2900000,chalet,800.0,FALSE,8,7,La Moraleja,La Moraleja,La Moraleja,40.5121,-3.6235,12572,good,FALSE,FALSE,3625,TRUE,TRUE,Chueca-Justicia,3,FALSE,NaN,FALSE
4,87535127,430000,flat,93.0,TRUE,1,2,"Paseo del Prado, 14",Madrid,Centro,40.4142,-3.6944,795,good,FALSE,FALSE,4624,TRUE,TRUE,Huertas-Cortes,4,TRUE,NaN,FALSE


In [15]:
# Total amount of unique Property Codes
total = len(df.index)

print('There are {} total unique records.'.format(total))

There are 6572 total unique records.


In [22]:
######################################################
# Save cleaned DataFrame without Duplicates as a CSV #
#df.to_csv('ALL-JSON-FILES.csv')                      #
######################################################

## Analysis

In [17]:
# Value counts of column
df['floor'].value_counts()

1      1414
2      1172
bj     1071
3      1021
4       814
5       380
6       220
7       151
en       93
8        62
9        42
ss       35
10       22
12       16
11       15
14       14
st       11
-1       10
15        3
nan       2
17        2
13        2
Name: floor, dtype: int64

In [18]:
df['status'].value_counts()

good              5298
renew             1175
newdevelopment      99
Name: status, dtype: int64

In [19]:
dictionary = df['municipality'].value_counts().to_dict()
dictionary

{'Alcalá de Henares': 80,
 'Alcobendas': 26,
 'Alcorcón': 74,
 'Algete': 11,
 'Arganda': 32,
 'Arroyomolinos': 28,
 'Boadilla del Monte': 51,
 'Brunete': 12,
 'Camarma de Esteruelas': 1,
 'Campo Real': 5,
 'Casarrubuelos': 4,
 'Chinchón': 1,
 'Ciempozuelos': 24,
 'Ciudalcampo': 15,
 'Cobeña': 3,
 'Colmenar Viejo': 25,
 'Colmenarejo': 3,
 'Coslada': 20,
 'Cubas de la Sagra': 10,
 'Daganzo de Arriba': 5,
 'Fuenlabrada': 125,
 'Fuente del Fresno': 2,
 'Fuente el Saz de Jarama': 7,
 'Galapagar': 8,
 'Getafe': 99,
 'Griñón': 10,
 'Hoyo de Manzanares': 6,
 'Humanes de Madrid': 36,
 'La Moraleja': 107,
 'Las Rozas de Madrid': 88,
 'Leganés': 143,
 'Loeches': 11,
 'Madrid': 4584,
 'Majadahonda': 51,
 'Mejorada del Campo': 28,
 'Moraleja de Enmedio': 4,
 'Morata de Tajuña': 1,
 'Móstoles': 101,
 'Navalcarnero': 26,
 'Paracuellos de Jarama': 8,
 'Parla': 90,
 'Pinto': 22,
 'Pozuelo de Alarcón': 137,
 'Quijorna': 8,
 'Rivas-Vaciamadrid': 27,
 'San Agustin de Guadalix': 10,
 'San Fernando de Henar

In [20]:
dictionary = df['neighborhood'].value_counts().to_dict()
dictionary

{'12 de Octubre-Orcasur': 15,
 'Abrantes': 58,
 'Acacias': 46,
 'Adelfas': 16,
 'Alameda de Osuna': 5,
 'Almagro': 74,
 'Almendrales': 39,
 'Alto de la Jabonería': 1,
 'Aluche': 72,
 'Ambroz': 30,
 'Amposta': 9,
 'Apóstol Santiago': 16,
 'Arapiles': 42,
 'Aravaca': 51,
 'Arcos': 49,
 'Argüelles': 46,
 'Arroyo del Fresno': 1,
 'Atalaya': 57,
 'Batallas': 18,
 'Bellas Vistas': 70,
 'Bernabéu-Hispanoamérica': 45,
 'Berruguete': 66,
 'Buena Vista': 61,
 'Butarque': 27,
 'Campamento': 18,
 'Campo de Tiro': 1,
 'Campo de las Naciones-Corralejos': 2,
 'Campodón - Ventorro del Cano': 3,
 'Canillas': 48,
 'Canillejas': 17,
 'Casa de Campo': 12,
 'Casco Antiguo': 86,
 'Casco Histórico de Barajas': 15,
 'Casco Histórico de Vallecas': 31,
 'Casco Histórico de Vicálvaro': 11,
 'Castellana': 61,
 'Castilla': 28,
 'Chopera': 67,
 'Chueca-Justicia': 108,
 'Ciudad Jardín': 26,
 'Ciudad Universitaria': 55,
 'Colina': 6,
 'Comillas': 85,
 'Concepción': 25,
 'Conde Orgaz-Piovera': 50,
 'Costillares': 25,


In [21]:
# Replace specific values with something else
#df['floor'] = df['floor'].replace(['-1', 'ss', 'bj'], 'bajo')
#df['floor'] = df['floor'].replace(['1'], 'normal')
#...